In [ ]:
!pip install --upgrade langchain openai -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils
!pip install chromadb
!pip install sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
import os
import openai
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.llms import OpenAI
import os

In [ ]:
# load pdfs or txt files from data folder
directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

2

In [ ]:
# split the data in chunks with overlap
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [ ]:
docs = split_docs(documents)
print(len(docs))

9


In [ ]:
# get embeddings for data
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
query_result = embeddings.embed_query("Hello world")
len(query_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


384

In [ ]:
# use chromadb to store the embeddings
index  = Chroma.from_documents(docs, embeddings)

In [ ]:
index

In [ ]:
# return top 2 queries based on similarity score
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

In [ ]:
os.environ["OPENAI_API_KEY"] = "<KEY>"
os.environ["HUGGINGFACEHUB_API_TOKEN"] ="<KEY>"

In [ ]:
# load llm
llm = HuggingFaceHub( repo_id="google/flan-ul2")
# define qa langchain
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [ ]:
# this is correct
query = "how much is india's economy?"
answer = get_answer(query)
print(answer)

$3.6 trillion


In [ ]:
# this is correct
query = "how much is usa economy?"
answer = get_answer(query)
print(answer)

$25.3 trillion


In [ ]:
# ratio was expected but relevant text was returned
query = "ratio of usa economy compared to india?"
answer = get_answer(query)
print(answer)

The US, with a GDP of $25.3 trillion (2023 est. ), reigns supreme on the world stage. India, at $3.6 trillion, occupies the fifth position, trailing China, Japan, and Germany.


In [ ]:
# this time ratio was provided
query = "ratio of usa's gdp per capita compared to india?"
answer = get_answer(query)
print(answer)

75000:3000


In [ ]:
# this is correct
query = "what is usa service sector contribution to its gdp?"
answer = get_answer(query)
print(answer)

81%


In [ ]:
# this is correct
query = "what is india service sector contribution to its gdp?"
answer = get_answer(query)
print(answer)

52%


In [ ]:
# ratio was expected, but text has been returned
query = "what will ratio of service sector contribution to gdp for india compared to usa? i want ratio"
answer = get_answer(query)
print(answer)

India finds a more balanced approach with service (52%), manufacturing (17.3%), and agriculture (18.5%) contributing significantly
